In [ ]:
# Import Packages
using Pkg  # Package to install new packages

# Install packages 
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("Plots")
#Pkg.add("Lathe")
#Pkg.add("GLM")
#Pkg.add("StatsPlots")
#Pkg.add("MLBase")
#Pkg.add("flux")


# Load the installed packages
using DataFrames
using CSV
using Plots
using Lathe
using GLM
using Statistics
using StatsPlots
using MLBase
using Printf
using Dates
using Flux: onehot
using Flux: onehotbatch
using Query
using VegaLite
using DataVoyager
using FileIO
using ElectronDisplay

# Enable printing of 1000 columns
ENV["COLUMNS"] = 1000

In [ ]:
data = DataFrame(CSV.File("../Datasets/sleepdata.csv"))
dropmissing!(data)
rename!(data, :"Sleep quality" => "SleepQuality", :"Time in bed" => "Time_in_bed", "Wake up" => "Wake_up", :"Sleep Notes" => "Sleep_Notes", :"Heart rate" => "Heart_rate", :"Activity (steps)" => "Activity_(steps)")
first(data,5)

In [ ]:
data.Start = Date.(data.Start, "yyyy-mm-dd HH:MM:SS");
data.End = Date.(data.End, "yyyy-mm-dd HH:MM:SS");
first(data, 5)

In [ ]:
const nano = 60000000000 # how many nanoseconds are in a minute

insertcols!(data, :Minutes_in_bed => 1)
for i in range(1, length=nrow(data))
    data.Minutes_in_bed[i] = Dates.value(data.Time_in_bed[i]) / nano
end

In [ ]:
first(data, 5)

In [ ]:
for i in range(1, length=nrow(data))
    if occursin(":", data.Sleep_Notes[i])
        data.Sleep_Notes[i] = split(data.Sleep_Notes[i], ":" )[1]
    end
end

In [304]:
first(data, 5)

,Start,End,SleepQuality,Time_in_bed,Wake_up,Sleep_Notes,Heart_rate,Activity_(steps),Minutes_in_bed,Stressful_day,Drank_coffee,Drank_tea,Worked_out,Ate_late
,Date,Date,String,Time,String,String,Int64,Int64,Int64,Bool,Bool,Bool,Bool,Bool
1,2014-12-30,2014-12-30,3%,00:16:00,:|,Stressful day,72,0,16,1,0,0,0,0
2,2015-01-01,2015-01-02,72%,06:44:00,:),Drank coffee,68,0,404,0,1,0,0,0
3,2015-01-03,2015-01-03,83%,07:12:00,:),Drank coffee,60,0,432,0,1,0,0,0
4,2015-01-04,2015-01-05,78%,07:18:00,:),Ate late,57,0,438,0,0,0,0,1
5,2015-01-05,2015-01-06,69%,07:27:00,:),Drank coffee,56,0,447,0,1,0,0,0


In [336]:


a = onehotbatch(["Ate late", "Drank tea"], ["Stressful day", "Drank coffee", "Drank tea", "Worked out", "Ate late"])

a = a[:, 1] + a[:, 2]

5-element Vector{Int64}:
 0
 0
 1
 0
 1

In [305]:
sleepnotes = ["Stressful_day", "Drank_coffee", "Drank_tea", "Worked_out", "Ate_late"]

ohe = onehot(data.Sleep_Notes[1], ["Stressful day", "Drank coffee", "Drank tea", "Worked out", "Ate late"])

for i in range(2, length = nrow(data) - 1)
    nextOHE = onehot(data.Sleep_Notes[i], ["Stressful day", "Drank coffee", "Drank tea", "Worked out", "Ate late"])
    ohe = hcat(ohe, nextOHE)
end

Bool[1, 0, 0, 0, 0]

In [ ]:
oheDF = convert(DataFrame, transpose(ohe))

rename!(oheDF, :"x1" => "Stressful_day", :"x2" => "Drank_coffee", "x3" => "Drank_tea", :"x4" => "Worked_out", :"x5" => "Ate_late")

data = hcat(data, oheDF)

In [317]:
first(data, 10)


,Start,End,SleepQuality,Time_in_bed,Wake_up,Sleep_Notes,Heart_rate,Activity_(steps),Minutes_in_bed,Stressful_day,Drank_coffee,Drank_tea,Worked_out,Ate_late
,Date,Date,String,Time,String,String,Int64,Int64,Int64,Bool,Bool,Bool,Bool,Bool
1,2014-12-30,2014-12-30,3%,00:16:00,:|,Stressful day,72,0,16,1,0,0,0,0
2,2015-01-01,2015-01-02,72%,06:44:00,:),Drank coffee,68,0,404,0,1,0,0,0
3,2015-01-03,2015-01-03,83%,07:12:00,:),Drank coffee,60,0,432,0,1,0,0,0
4,2015-01-04,2015-01-05,78%,07:18:00,:),Ate late,57,0,438,0,0,0,0,1
5,2015-01-05,2015-01-06,69%,07:27:00,:),Drank coffee,56,0,447,0,1,0,0,0
6,2015-01-06,2015-01-07,74%,07:35:00,:|,Drank tea,64,0,455,0,0,1,0,0
7,2015-01-07,2015-01-08,81%,09:19:00,:),Drank coffee,62,0,559,0,1,0,0,0
8,2015-01-08,2015-01-09,88%,07:16:00,:),Drank coffee,58,0,436,0,1,0,0,0
9,2015-01-09,2015-01-10,77%,09:01:00,:|,Drank coffee,65,0,541,0,1,0,0,0
